# Ley de los grandes números

La ley de los grandes números afirma que mientras un experimento aleatorio se repita una gran cantidad de veces $n$, el promedio de los resultados $\frac{X_1, X_2, \ldots, X_n}{n}$ de ese experimento se aproximará a su valor esperado $E(X)$ conforme $n$ se aproxime a $+\infty$.

Los ejemplos más usados para demostrar la ley de los grandes números consisten en lanzar una moneda al aire o la tirada de un dado de seis caras.

Para el lanzamiento de una moneda, el $E(X) = 0.5$. Los lenguajes computacionales permiten experimentar computacionalmente mediante la obtencion de [valores pseudoaleatorios](https://es.wikipedia.org/wiki/Generador_de_n%C3%BAmeros_pseudoaleatorios). En [R](https://www.r-project.org/) se puede realizar un experimento que simule mil lanzamientos de monedas para, de ellos, calcular la media, con el procedimiento

In [164]:
set.seed(33)
mean(sample(0:1, 1000, replace=TRUE))

[1] 0.507

que da por resultado $0.503 \approx E(X)$. En este caso, ese `0:1` simula el conjunto de resultados`0` para cara y `1` para cruz. En el caso de la tirada del dado, se puede simular así:

In [165]:
mean(sample(1:6, 1000, replace=TRUE))

[1] 3.554

donde el valor $E(X) = \frac{1 + 2 + 3 + 4 + 5 + 6}{6} = 2.5 \approx 3.524$.

Una representación gráfica de la parte en la que $n$ se aproxima a $+\infty$ consiste en graficar las medias conforme $n$ crece. Para el ejemplo de la tirada del dado de seis cara, se logra mediante:

In [20]:
medias <- c()
for (i in 1:10000){
    medias <- c(medias, mean(sample(1:6, i, replace=TRUE)))
}

In [167]:
pdf('dados.pdf')
plot(medias, type='l', ylab="Medias", xlab="Repeticiones", cex.lab = 1.4, cex.axis = 1.4)
dev.off()

png 
  2

Actualmente, investigo la relación que hay entre los contaminantes del aire y los reportes de enfermedades por parte de los centros de salud, estudio delimitado al Área Metropolitana de Monterrey durante el año 2017. Los datos de contaminantes para ese año se obtuvieron del [Sistema Integral de Monitoreo Ambiental](http://aire.nl.gob.mx/).

Primero se leen los datos.

In [172]:
datos <- read.csv('all.csv')

Este es un ejemplo de los datos obtenidos.

In [168]:
library(xtable)

In [208]:
set.seed(94)
tabla = datos[sample(nrow(datos), 10), ]
print(xtable(tabla), include.rownames = TRUE)

% latex table generated in R 4.0.2 by xtable 1.8-4 package
% Tue Dec 01 00:11:29 2020
\begin{table}[ht]
\centering
\begin{tabular}{rllrrrrrrrrrrrrrrrrl}
  \hline
 & timestamp & station & CO & NO & NO2 & NOX & O3 & PM10 & PM2\_5 & pressure & rainfall & humidity & SO2 & solar & temperature & velocity & direction & valid & notes \\ 
  \hline
1018306 & 29-Jul-13 11 & Noreste2 & 0.44 & 4.00 & 1.40 & 5.40 & 49.00 & 83.00 &  & 719.50 & 0.00 & 64.00 & 6.50 & 0.81 & 29.41 & 4.70 & 92.00 &   1 &  \\ 
  761468 & 25-Dec-09 6 & Noreste & 0.18 & 2.70 & 8.40 & 10.90 & 22.00 & 14.00 & 8.00 &  &  & 50.00 & 3.30 &  & 5.94 & 5.30 & 316.00 &   1 &  \\ 
  623230 & 22-Mar-07 13 & Suroeste & 1.36 & 12.50 & 23.40 & 35.50 & 30.00 & 114.00 &  & 700.90 & 0.00 & 68.00 & 6.80 &  & 21.86 & 5.40 & 137.00 &   1 &  \\ 
  850935 & 21-May-11 16 & Norte & 3.44 & 2.20 & 6.30 & 8.20 & 57.00 & 20.00 & 8.00 & 714.20 & 0.00 & 42.00 & 2.30 & 0.62 & 35.16 & 15.40 & 103.00 &   1 &  \\ 
  431462 & 05-Nov-02 12 & Noreste & 0.70 & 

La primera columna es una fecha, así que se convierte en un formato de fecha que interpreta R:

In [47]:
# https://stackoverflow.com/a/32068524/3113008
# https://stackoverflow.com/a/29197887/3113008
Sys.setlocale("LC_ALL","English")

datos$timestamp = as.Date(tolower(datos$timestamp), format = '%d-%b-%y %H')

[1] "LC_COLLATE=English_United States.1252;LC_CTYPE=English_United States.1252;LC_MONETARY=English_United States.1252;LC_NUMERIC=C;LC_TIME=English_United States.1252"

Para esta tarea, nos intersa estimar los valores inválidos que se presentarán por mes, año y estación, así que primero se resta $1$ a todos los valores de la columna `valid`, para que los valores inválidos tengan un valor de `-1`  y los válidos de `0`.

In [56]:
datos$valid = datos$valid - 1

Ahora se pueden agrupar por mes, año y estación.

In [60]:
library(dplyr)

In [138]:
errores = datos %>%
    mutate(month = format(timestamp, "%m"), year = format(timestamp, "%Y")) %>%
    group_by(year, station) %>%
    summarise(errores = abs(sum(valid)), total = n())

`summarise()` regrouping output by 'year' (override with `.groups` argument)



In [211]:
print(xtable(errores[sample(nrow(errores), 3), ]))

% latex table generated in R 4.0.2 by xtable 1.8-4 package
% Tue Dec 01 00:26:31 2020
\begin{table}[ht]
\centering
\begin{tabular}{rllrr}
  \hline
 & year & station & errores & total \\ 
  \hline
1 & 1999 & Noroeste & 40.00 & 8760 \\ 
  2 & 2003 & Suroeste & 0.00 & 8760 \\ 
  3 & 2000 & Centro & 35.00 & 8784 \\ 
   \hline
\end{tabular}
\end{table}


In [140]:
estaciones = unique(errores$station)
estaciones

[1] "Centro"    "Noreste"   "Noroeste"  "Sureste"   "Suroeste"  "Noroeste2"
 [7] "Norte"     "Noreste2"  "Sureste2"  "Suroeste2" "Norte2"    "Sur"      
[13] "Sureste3"

In [162]:
# for (e in estaciones){
#     t = errores[errores$station == e, ]
#     plot(t$year, t$errores / t$total, xlab="Año", ylab="Total de errores", type='l')
#     cat(e, ' ', sum(t$errores / t$total), '\n')
# }

Del análisis anterior, se ve que la estación Suroeste es la que más datos inválidos registra (un $41.47\%$ de los datos reportados):

In [212]:
t = errores[errores$station == 'Suroeste', ]
pdf('so.pdf')
plot(t$year, t$errores / t$total, xlab="Año", ylab="Porcentaje de errores", type='l', cex.lab = 1.4, cex.axis = 1.4)
dev.off()
cat('Suroeste', ' ', sum(t$errores / t$total), '\n')

png 
  2

Suroeste   0.4147042 


Por medio de la ley de los grandes números se puede calcular la cantidad de mediciones que se deben hacer para saber, con un $95\%$ de confianza, que ese porcentaje de errores de medición es correcto. En el caso del sensor del suroeste, se sabe que la probabilidad de obtener una medición inválida es $\mu = 0.4147 = p$, de donde $\sigma^2 = p(1 - p) = 0.2427$. Ahora, se puede definir un error de medición $\epsilon = 0.02$, lo que quiere decir que se probará si una medida sea inválida con probabilidad entre $[0.4147 - 0.02, 0.4147 + 0.02]$. Por la ley de los grandes números, se tiene que
$$P[|\bar{X} - \mu| > \epsilon] \leq \frac{\sigma^2}{n \epsilon^2},$$ 
al sustituir
$$P[|\bar{X} - 0.4147| > 0.02] \leq \frac{0.2427^2}{n (0.02)^2}.$$
Como se desea obtener un intervalo de confianza de $95%$, entonces se debe cumplir
$$\frac{0.2427^2}{n (0.02)^2} = 0.05,$$
por lo que
$$n = \frac{0.2427^2}{(0.05) (0.02)^2} = 2945.1645 \approx 2945.$$

Así, se podría recomendar realizar $2945$ mediciones